Data Ingestion

In [24]:
import pandas as pd

csv_file_path = './archive/Albania_COVID19.csv'
df = pd.read_csv(csv_file_path)

# Display the DataFrame
df.head()

,Date,State,Country,Cumulative_cases,Cumulative_death,Daily_cases,Daily_death,Latitude,Longitude,Temperature,...,Available Beds/1000,Confirmed Cases/1000,Lung Patients (F),Lung Patients (M),Life Expectancy (M),Life Expectancy (F),Total_tests_conducted,Out_Travels (mill.),In_travels(mill.),Domestic_Travels (mill.)
0,22-01-2020,NaN,Albania,0,0,0,0,41.1533,20.1683,5.72,...,0.725,0.0,7.02,17.04,76,81.6,1526,5415,5927,Not Reported
1,23-01-2020,NaN,Albania,0,0,0,0,41.1533,20.1683,6.72,...,0.725,0.0,7.02,17.04,76,81.6,1526,5415,5927,Not Reported
2,24-01-2020,NaN,Albania,0,0,0,0,41.1533,20.1683,8.17,...,0.725,0.0,7.02,17.04,76,81.6,1526,5415,5927,Not Reported
3,25-01-2020,NaN,Albania,0,0,0,0,41.1533,20.1683,10.83,...,0.725,0.0,7.02,17.04,76,81.6,1526,5415,5927,Not Reported
4,26-01-2020,NaN,Albania,0,0,0,0,41.1533,20.1683,11.39,...,0.725,0.0,7.02,17.04,76,81.6,1526,5415,5927,Not Reported


CREATE TABLE QUERY

In [25]:
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SAWarning
import warnings

# Suppress the SAWarning
warnings.filterwarnings('ignore', category=SAWarning)

server = 'HP\SQLEXPRESS'
database = 'python'
username = 'HP\91760'
password = ''
driver = 'ODBC Driver 17 for SQL Server'

# Create a connection string
connection_string = f'SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Establish a connection
engine = create_engine(f'mssql+pyodbc://{server}/{database}?DRIVER={driver}&{connection_string}')

# Create a table named db_1
table_name = 'db_1'

# Write DataFrame to SQL Server with a dynamic table name
df.to_sql(table_name, engine, index=False, if_exists='replace')

# Display the DataFrame
print(f'DataFrame has been written to the SQL table: {table_name}')

DataFrame has been written to the SQL table: db_1


UPDATE QUERY

In [26]:
# Update queries

# Set the new value in Cumulative_cases column for specific conditions
condition_1 = (df['Country'] == 'Albania') & (df['Date'] == '22-01-2020')
df.loc[condition_1, 'Cumulative_cases'] = 100

condition_2 = (df['Country'] == 'Albania') & (df['Date'] == '23-01-2020')
df.loc[condition_2, 'Confirmed_cases'] = 100

condition_3 = (df['Country'] == 'France') & (df['Date'] == '22-01-2020')
df.loc[condition_3, 'Cumulative_cases'] = 150

condition_4 = (df['Country'] == 'France') & (df['Date'] == '23-01-2020')
df.loc[condition_4, 'Cumulative_cases'] = 200

# Display the Updated DataFrame
print(f'Updated DataFrame has been written to the SQL table: {table_name}')

Updated DataFrame has been written to the SQL table: db_1


FILTER QUERY

In [27]:
# Filter the table based on a country
filter_table_query = f'SELECT Cumulative_cases FROM {table_name} WHERE Country = \'Albania\';'
filtered_df = pd.read_sql_query(filter_table_query, engine)

print(f'Filtered DataFrame:\n{filtered_df}')

# More filter conditions
filter_table_query_2 = f'SELECT * FROM {table_name} WHERE Country IN (\'Albania\', \'France\');'
filtered_df_2 = pd.read_sql_query(filter_table_query_2, engine)

print(f'Filtered DataFrame:\n{filtered_df_2}')

Filtered DataFrame:
    Cumulative_cases
0                  0
1                  0
2                  0
3                  0
4                  0
..               ...
64               174
65               186
66               197
67               212
68               223

[69 rows x 1 columns]
Filtered DataFrame:
          Date State  Country  Cumulative_cases  Cumulative_death  \
0   22-01-2020  None  Albania                 0                 0   
1   23-01-2020  None  Albania                 0                 0   
2   24-01-2020  None  Albania                 0                 0   
3   25-01-2020  None  Albania                 0                 0   
4   26-01-2020  None  Albania                 0                 0   
..         ...   ...      ...               ...               ...   
64  26-03-2020  None  Albania               174                 6   
65  27-03-2020  None  Albania               186                 8   
66  28-03-2020  None  Albania               197                1

ALTER TABLE QUERY

In [28]:
# Alter table to add a new column
alter_table_query = f'ALTER TABLE {table_name} ADD New_Column INT;'
with engine.connect() as connection:
    connection.execute(text(alter_table_query))

print(f'Table altered: {table_name}')


Table altered: db_1


INSERT QUERY

In [33]:
# Insert a new row into the table
insert_query = f"INSERT INTO {table_name} (Country, Date, Cumulative_cases, Daily_death) VALUES ('France', '24-01-2020', 50, 30);"
with engine.connect() as connection:
    connection.execute(text(insert_query))

print(f'Row inserted into the table: {table_name}')


Row inserted into the table: db_1


DELETE QUERY

In [ ]:
# Delete rows from the table based on a condition
delete_query = f"DELETE FROM {table_name} WHERE Country = 'France';"
with engine.connect() as connection:
    connection.execute(text(delete_query))

print(f'Rows deleted from the table: {table_name}')


DROP QUERY

In [ ]:
# Drop the table
with engine.connect() as connection:
    drop_statement = text(f'DROP TABLE {table_name}')
    connection.execute(drop_statement)

print(f'Table dropped: {table_name}')